In [1]:
pip install json-repair

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
from langchain.docstore.document import Document
from langchain_experimental.graph_transformers.llm import UnstructuredRelation, examples
from langchain.schema import SystemMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models import ChatOllama
import time

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def recurs_split(x):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=400, chunk_overlap=50
    )
    split_docs = text_splitter.create_documents([x])
    return(split_docs)

In [19]:
llm = ChatOllama(temperature=0, model = "llama3")

In [20]:

system_prompt = """
You are a data scientist working for a company that is building a knowledge graph database. 
Your task is to extract information from data and convert it into a knowledge graph database.
Provide a set of Nodes in the form [head, head_type, relation, tail, tail_type].
It is important that the head and tail exists as nodes that are related by the relation. If you can't pair a relationship with a pair of nodes don't add it.
When you find a node or relationship you want to add try to create a generic TYPE for it that describes the entity you can also think of it as a label.
You must generate the output in a JSON format containing a list with JSON objects. Each object should have the keys: "head", "head_type", "relation", "tail", and "tail_type".
"""

system_message = SystemMessage(content=system_prompt)
parser = JsonOutputParser(pydantic_object=UnstructuredRelation)

human_prompt = PromptTemplate(
    template="""
Examples:
{examples}

For the following text, extract entities and relations as in the provided example.
{format_instructions}\nText: {input}""",
    input_variables=["input"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
        "node_labels": None,
        "rel_types": None,
        "examples": examples,
    },
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message, human_message_prompt]
)

In [21]:
llm_transformer = LLMGraphTransformer(llm=llm, prompt=chat_prompt)

In [22]:
text = "[BIOGRAPHY] To be determined. [IDENTIFICATION] Permanent resident of Malaysia. Alternative age: 55 (2016/01/10). [REPORTS] Jan 2016 - charged with human trafficking of an under aged girl for the purpose of labour exploitation. Pleaded not guilty. Remanded."

In [23]:
strr = [recurs_split(text)]

strr

[[Document(page_content='[BIOGRAPHY] To be determined. [IDENTIFICATION] Permanent resident of Malaysia. Alternative age: 55 (2016/01/10). [REPORTS] Jan 2016 - charged with human trafficking of an under aged girl for the purpose of labour exploitation. Pleaded not guilty. Remanded.')]]

In [25]:
# strr[0]

[Document(page_content='[BIOGRAPHY] To be determined. [IDENTIFICATION] Permanent resident of Malaysia. Alternative age: 55 (2016/01/10). [REPORTS] Jan 2016 - charged with human trafficking of an under aged girl for the purpose of labour exploitation. Pleaded not guilty. Remanded.')]

In [26]:
st1 = time.time()
graph_doc = llm_transformer.convert_to_graph_documents(strr[0])
et1 = time.time()

In [27]:
print(et1-st1)

10.94710898399353


In [28]:
print(graph_doc)

[GraphDocument(nodes=[Node(id='55', type='Age'), Node(id='human trafficking of an under aged girl for the purpose of labour exploitation', type='Crime'), Node(id='Malaysia', type='Location'), Node(id='To be determined', type='Person'), Node(id='Permanent resident of Malaysia', type='Location')], relationships=[Relationship(source=Node(id='To be determined', type='Person'), target=Node(id='Permanent resident of Malaysia', type='Location'), type='IDENTIFIED_AS'), Relationship(source=Node(id='To be determined', type='Person'), target=Node(id='55', type='Age'), type='HAS_AGE'), Relationship(source=Node(id='To be determined', type='Person'), target=Node(id='Malaysia', type='Location'), type='LIVES_IN'), Relationship(source=Node(id='To be determined', type='Person'), target=Node(id='human trafficking of an under aged girl for the purpose of labour exploitation', type='Crime'), type='CHARGED_WITH')], source=Document(page_content='[BIOGRAPHY] To be determined. [IDENTIFICATION] Permanent reside

In [29]:
text = "[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. POB: Kelantan, Malaysia. Citizenship: Malaysia. Address: Kota Bharu, Kelantan, Malaysia. Additional Information: Passport number A11457974, National identification number 670516-03-5283 In custody in the Philippines as of Feb. 2008. Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [CANADA SANCTIONS - UNSTR] May 2011 - amended. Mar 2013 - removed. QI.Z.127.03. PRIMARY NAME: Zakaria,Zaini. POB; Kelantan, Malaysia. DOB: 16/05/67. Nationality: Malaysian. Passport no.: A 11457974; National identification no.: 670516-03-5283; Address: Kota Bharu, State of Kelantan, Malaysia. [EU SANCTIONS] EC 881/2002 (Apr 2013 - removed). 2002/402/CFSP PRIMARY NAME: Zaini Zakaria (alias Ahmad). Address: Kota Bharu, Kelantan, Malaysia. Date of birth: 16.5.1967. Place of birth: Kelantan, Malaysia. Nationality: Malaysian. Passport No: A11457974. National identification No: 670516-03-5283. Date of designation referred to in Article 2a (4) (b): 9.9.2003. [HONG KONG SANCTIONS - HKGAZ] G.N.6664 (14 Oct 2011) (AQ/TA split). G.N.2219 (26 April 2013 - removed). QI.Z.127.03. PRIMARY NAME:ZAINI ZAKARIA. DOB: 16 May 1967. POB: Kelantan, Malaysia. Low quality a.k.a.: Ahmad. Nationality: Malaysian. Passport no.: A 11457974. National identification no.: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [ISRAEL SANCTIONS - IMOD] Nov 2011 - addition. May 2013 - removed. Listed under Section 2 of Prohibition of Financing Terrorism Law. PRIMARY NAME: Zaini Zakaria. [SWITZERLAND SANCTIONS - SECO] SSID: 10-17045 (Sep 2003 - addition. Apr 2013 - removed). Foreign identifier: QI.Z.127.03. PRIMARY NAME: Zaini Zakaria DOB: 16 May 1967 POB: Kelantan, Malaysia Low quality a.k.a.: Ahmad Address: Kota Bharu, State of Kelantan, Malaysia Nationality: Malaysia Identification document: a) Passport No. A 11457974, Malaysia b) ID card No. 670516-03-5283, Malaysia Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun 2009. [UK SANCTIONS - UKHMT] Sep 2003 - addition. Apr 2013 - removed. PRIMARY NAME: ZAKARIA,ZAINI. DOB: 16/05/1967. POB: Kelantan, Malaysia. Nationality: Malaysian. Passport Details: A 11457974. National Identification no: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other Information: UN Ref QI.Z.127.03. Also referred to as Ahmad. Group ID: 7852. [UN SANCTIONS] UNSCR 1267 (1999) (Sep 2003 - addition. Mar 2013 - removed). QI.Z.127.03. PRIMARY NAME:ZAINI ZAKARIA. DOB: 16 May 1967. POB: Kelantan, Malaysia. Low quality a.k.a.: Ahmad. Nationality: Malaysian. Passport no.: A 11457974. National identification no.: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [UN REPORTS - INTERPOL] 2006-25701. No longer appears on Interpol-United Nations Security Council Special Notice, subject to UN Sanctions - Al-Qaida (QI.Z.127.03.). [USA SANCTIONS - OFAC] SDN Ref No 7939 - SDGT (Specially Designated Global Terrorist) (Sep 2003 - addition. Sep 2009 - amended). PRIMARY NAME: ZAKARIA, Zaini. A-1-8 Pangsapuri Sri Tanjung, Jalan Sky BS2, Bukit Sentosa Rawang, Selangor 48300, Malaysia. DOB 16 Jun 1967; alt. DOB 16 May 1967. POB Kelantan, Malaysia. Passport A11457974 (Malaysia). [SANCTIONS NOTE] This entry has been removed by a published EU regulation. [SANCTIONS HISTORY] EU (Regulation)(- Apr 2013). UKHMT (Sep 2003 - Apr 2013). UN (Sep 2003 - Mar 2013). Mar 2013 - removed from Australia, Canada, China, Jersey, New Zealand, Singapore and Ukraine sanctions lists. Apr 2013 - removed from Austria, France (Regulation), Germany (Regulation), Hong Kong (HKGAZ), India, Isle of Man, Luxembourg, Malta, Russia, Spain (Regulation) and Switzerland (SECO) sanctions lists. May 2013 - removed from Israel and Japan (JMOF) sanctions lists. Sep 2013 - removed from Monaco sanctions list. Oct 2013 - removed from Azerbaijan sanctions list. Feb 2016 - removed from South Korea (KRMSF) sanctions list. Feb 2017 - no longer appears on Liechtenstein sanctions list. [BIOGRAPHY] Alleged member {General} of Jemaa Islamiyah (JI). Reportedly recruited by al Qa'ida to pilot a plane in a second wave of 9/11 style attacks on the United States. [IDENTIFICATION] Resident of Kota Bharu, Kelantan. Abdul Matin Anol Rahmat (father-in-law). Also referred to as AHMAD. [FUNDING] Engineer. [REPORTS] May 2013 - removed by the Israeli Ministerial Committee for National Security Affairs from the Individuals sanction list. Nov 2020 - reportedly subject to export control measures related to Strategic High-Tech Commodities (SHTC) imposed by Taiwan's Bureau of Foreign Trade (BFT).[NONCONVICTION TERROR CATEGORY NOTICE] This category includes information about individuals who are reportedly being investigated for, or have been arrested or charged on suspicion of, involvement in terrorism or terror related activities. The category also includes individuals or entities identified on national or internationally recognised banning, warning or wanted lists as allegedly connected to terrorism or individuals who are reportedly connected to an organisation included on any national or international terrorism list. Inclusion in this category does not mean that an individual or entity is a terrorist or terrorist organisation or that they are involved in or connected to terrorism or terror related activity. Individuals included in this category have not been convicted of any terror related activity; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the outcome of any alleged investigation, arrest, charges or any reported connection to any terror related activity and whether such allegations are denied."

In [30]:
strr2 = [recurs_split(text)]

strr2

st1 = time.time()
graph_doc2 = llm_transformer.convert_to_graph_documents(strr2[0])
et1 = time.time()

In [31]:
print(et1-st1)

117.37574505805969


In [32]:
print(graph_doc2)

[GraphDocument(nodes=[Node(id='A11457974', type='Passport Number'), Node(id='16.5.1967', type='Date of Birth'), Node(id='EU SANCTIONS', type='Sanctions'), Node(id='CANADA SANCTIONS', type='Sanctions'), Node(id='Zaini Zakaria', type='Person'), Node(id='670516-03-5283', type='National Identification Number'), Node(id='Kota Bharu, Kelantan, Malaysia', type='Address'), Node(id='Kota Bharu, State of Kelantan, Malaysia', type='Address'), Node(id='Zakaria, Zaini', type='Person'), Node(id='HONG KONG SANCTIONS', type='Sanctions'), Node(id='Kelantan, Malaysia', type='Place'), Node(id='Ahmad', type='Alias'), Node(id='AUSTRALIA SANCTIONS', type='Sanctions'), Node(id='Philippines', type='Country')], relationships=[Relationship(source=Node(id='Zaini Zakaria', type='Person'), target=Node(id='AUSTRALIA SANCTIONS', type='Sanctions'), type='IS_SANCTIONED_BY'), Relationship(source=Node(id='Zaini Zakaria', type='Person'), target=Node(id='Ahmad', type='Alias'), type='HAS_ALIAS'), Relationship(source=Node(i

# PLEASE RUN TILL HERE

In [5]:
text = """[US DEPARTMENT OF COMMERCE] Bureau of Industry and Security - Appropriate Federal Register Citations: 73 F.R. 63678 10/27/08. Name - AHMED, YASMIN 612 BUSINESS CENTRE, MUMTAZ HASAN ROAD OFF I.I. CHUNDRIGAR ROAD, KARACHI, PK. Denied Persons List effective term 16 Oct 2008 - 16 Oct 2015. Oct 2015 - denial order expired. [US STATE DEPARTMENT] Directorate of Defence Trade Controls list of Parties Debarred for Arms Export Control Act convictions. 69 FR 17468, 04/02/2004. PRIMARY NAME - Ahmed, Yasmin (a.k.a. Yasmin Tariq; Fatimah Mohammad). [REGULATIONS HISTORY] Oct 2015 - denial order expired. [BIOGRAPHY] To be determined. [IDENTIFICATION] Naturalized US citizen. Worked in United Arab Emirates (reported 2003). Tariq Ahmed (spouse)."""

print(text)

[US DEPARTMENT OF COMMERCE] Bureau of Industry and Security - Appropriate Federal Register Citations: 73 F.R. 63678 10/27/08. Name - AHMED, YASMIN 612 BUSINESS CENTRE, MUMTAZ HASAN ROAD OFF I.I. CHUNDRIGAR ROAD, KARACHI, PK. Denied Persons List effective term 16 Oct 2008 - 16 Oct 2015. Oct 2015 - denial order expired. [US STATE DEPARTMENT] Directorate of Defence Trade Controls list of Parties Debarred for Arms Export Control Act convictions. 69 FR 17468, 04/02/2004. PRIMARY NAME - Ahmed, Yasmin (a.k.a. Yasmin Tariq; Fatimah Mohammad). [REGULATIONS HISTORY] Oct 2015 - denial order expired. [BIOGRAPHY] To be determined. [IDENTIFICATION] Naturalized US citizen. Worked in United Arab Emirates (reported 2003). Tariq Ahmed (spouse).


In [5]:
text = '[USA SANCTIONS - OFAC] SDN Ref No 31939 - OFAC-BM-EO14014 (Executive Order 14014) (May 2021 - addition). PRIMARY NAME: HTET, Kaung (a.k.a. HTET, U Kaung), Burma; DOB 03 Aug 1992; nationality Burma; Gender Male; National ID No. 12DAGANAN032517 (Burma). [BIOGRAPHY] To be determined. [IDENTIFICATION] National ID no: 12DAGANAN032517 (Burma). Alternative Citizenship: Myanmar. Maung Maung Kyaw (PEP) (father). Aung Mar Myint (mother). Kaung Htet (brother). Lin Lett Thiri (sister-in-law). [REPORTS] Sep 2021 - no further information reported.'

In [7]:
len(text)

539

In [11]:
import time

In [5]:
text =  "[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. POB: Kelantan, Malaysia. Citizenship: Malaysia. Address: Kota Bharu, Kelantan, Malaysia. Additional Information: Passport number A11457974, National identification number 670516-03-5283 In custody in the Philippines as of Feb. 2008. Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [CANADA SANCTIONS - UNSTR] May 2011 - amended. Mar 2013 - removed. QI.Z.127.03. PRIMARY NAME: Zakaria,Zaini. POB; Kelantan, Malaysia. DOB: 16/05/67. Nationality: Malaysian. Passport no.: A 11457974; National identification no.: 670516-03-5283; Address: Kota Bharu, State of Kelantan, Malaysia. [EU SANCTIONS] EC 881/2002 (Apr 2013 - removed). 2002/402/CFSP PRIMARY NAME: Zaini Zakaria (alias Ahmad). Address: Kota Bharu, Kelantan, Malaysia. Date of birth: 16.5.1967. Place of birth: Kelantan, Malaysia. Nationality: Malaysian. Passport No: A11457974. National identification No: 670516-03-5283. Date of designation referred to in Article 2a (4) (b): 9.9.2003. [HONG KONG SANCTIONS - HKGAZ] G.N.6664 (14 Oct 2011) (AQ/TA split). G.N.2219 (26 April 2013 - removed). QI.Z.127.03. PRIMARY NAME:ZAINI ZAKARIA. DOB: 16 May 1967. POB: Kelantan, Malaysia. Low quality a.k.a.: Ahmad. Nationality: Malaysian. Passport no.: A 11457974. National identification no.: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [ISRAEL SANCTIONS - IMOD] Nov 2011 - addition. May 2013 - removed. Listed under Section 2 of Prohibition of Financing Terrorism Law. PRIMARY NAME: Zaini Zakaria. [SWITZERLAND SANCTIONS - SECO] SSID: 10-17045 (Sep 2003 - addition. Apr 2013 - removed). Foreign identifier: QI.Z.127.03. PRIMARY NAME: Zaini Zakaria DOB: 16 May 1967 POB: Kelantan, Malaysia Low quality a.k.a.: Ahmad Address: Kota Bharu, State of Kelantan, Malaysia Nationality: Malaysia Identification document: a) Passport No. A 11457974, Malaysia b) ID card No. 670516-03-5283, Malaysia Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun 2009. [UK SANCTIONS - UKHMT] Sep 2003 - addition. Apr 2013 - removed. PRIMARY NAME: ZAKARIA,ZAINI. DOB: 16/05/1967. POB: Kelantan, Malaysia. Nationality: Malaysian. Passport Details: A 11457974. National Identification no: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other Information: UN Ref QI.Z.127.03. Also referred to as Ahmad. Group ID: 7852. [UN SANCTIONS] UNSCR 1267 (1999) (Sep 2003 - addition. Mar 2013 - removed). QI.Z.127.03. PRIMARY NAME:ZAINI ZAKARIA. DOB: 16 May 1967. POB: Kelantan, Malaysia. Low quality a.k.a.: Ahmad. Nationality: Malaysian. Passport no.: A 11457974. National identification no.: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [UN REPORTS - INTERPOL] 2006-25701. No longer appears on Interpol-United Nations Security Council Special Notice, subject to UN Sanctions - Al-Qaida (QI.Z.127.03.). [USA SANCTIONS - OFAC] SDN Ref No 7939 - SDGT (Specially Designated Global Terrorist) (Sep 2003 - addition. Sep 2009 - amended). PRIMARY NAME: ZAKARIA, Zaini. A-1-8 Pangsapuri Sri Tanjung, Jalan Sky BS2, Bukit Sentosa Rawang, Selangor 48300, Malaysia. DOB 16 Jun 1967; alt. DOB 16 May 1967. POB Kelantan, Malaysia. Passport A11457974 (Malaysia). [SANCTIONS NOTE] This entry has been removed by a published EU regulation. [SANCTIONS HISTORY] EU (Regulation)(- Apr 2013). UKHMT (Sep 2003 - Apr 2013). UN (Sep 2003 - Mar 2013). Mar 2013 - removed from Australia, Canada, China, Jersey, New Zealand, Singapore and Ukraine sanctions lists. Apr 2013 - removed from Austria, France (Regulation), Germany (Regulation), Hong Kong (HKGAZ), India, Isle of Man, Luxembourg, Malta, Russia, Spain (Regulation) and Switzerland (SECO) sanctions lists. May 2013 - removed from Israel and Japan (JMOF) sanctions lists. Sep 2013 - removed from Monaco sanctions list. Oct 2013 - removed from Azerbaijan sanctions list. Feb 2016 - removed from South Korea (KRMSF) sanctions list. Feb 2017 - no longer appears on Liechtenstein sanctions list. [BIOGRAPHY] Alleged member {General} of Jemaa Islamiyah (JI). Reportedly recruited by al Qa'ida to pilot a plane in a second wave of 9/11 style attacks on the United States. [IDENTIFICATION] Resident of Kota Bharu, Kelantan. Abdul Matin Anol Rahmat (father-in-law). Also referred to as AHMAD. [FUNDING] Engineer. [REPORTS] May 2013 - removed by the Israeli Ministerial Committee for National Security Affairs from the Individuals sanction list. Nov 2020 - reportedly subject to export control measures related to Strategic High-Tech Commodities (SHTC) imposed by Taiwan's Bureau of Foreign Trade (BFT).[NONCONVICTION TERROR CATEGORY NOTICE] This category includes information about individuals who are reportedly being investigated for, or have been arrested or charged on suspicion of, involvement in terrorism or terror related activities. The category also includes individuals or entities identified on national or internationally recognised banning, warning or wanted lists as allegedly connected to terrorism or individuals who are reportedly connected to an organisation included on any national or international terrorism list. Inclusion in this category does not mean that an individual or entity is a terrorist or terrorist organisation or that they are involved in or connected to terrorism or terror related activity. Individuals included in this category have not been convicted of any terror related activity; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the outcome of any alleged investigation, arrest, charges or any reported connection to any terror related activity and whether such allegations are denied."


In [6]:
splits = text

In [7]:
strr = [Document(text)]

In [8]:
strr

[Document(page_content="[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. POB: Kelantan, Malaysia. Citizenship: Malaysia. Address: Kota Bharu, Kelantan, Malaysia. Additional Information: Passport number A11457974, National identification number 670516-03-5283 In custody in the Philippines as of Feb. 2008. Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [CANADA SANCTIONS - UNSTR] May 2011 - amended. Mar 2013 - removed. QI.Z.127.03. PRIMARY NAME: Zakaria,Zaini. POB; Kelantan, Malaysia. DOB: 16/05/67. Nationality: Malaysian. Passport no.: A 11457974; National identification no.: 670516-03-5283; Address: Kota Bharu, State of Kelantan, Malaysia. [EU SANCTIONS] EC 881/2002 (Apr 2013 - removed). 2002/402/CFSP PRIMARY NAME: Zaini Zakaria (alias Ahmad). Address: Kota Bharu, Kelantan, Malaysia. Date of birth: 16.5.1967. Place of birth: Kelantan, Malaysia. Nationality: Malaysian. Passpor

In [9]:
# docs1 = [Document(page_content=t) for t in splits]

# docs1

In [12]:
st1 = time.time()
graph_doc = llm_transformer.convert_to_graph_documents(strr)
et1 = time.time()

In [13]:
print((et1-st1)/60)

11.392175356547037


In [14]:
graph_doc

[GraphDocument(nodes=[], relationships=[], source=Document(page_content="[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. POB: Kelantan, Malaysia. Citizenship: Malaysia. Address: Kota Bharu, Kelantan, Malaysia. Additional Information: Passport number A11457974, National identification number 670516-03-5283 In custody in the Philippines as of Feb. 2008. Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [CANADA SANCTIONS - UNSTR] May 2011 - amended. Mar 2013 - removed. QI.Z.127.03. PRIMARY NAME: Zakaria,Zaini. POB; Kelantan, Malaysia. DOB: 16/05/67. Nationality: Malaysian. Passport no.: A 11457974; National identification no.: 670516-03-5283; Address: Kota Bharu, State of Kelantan, Malaysia. [EU SANCTIONS] EC 881/2002 (Apr 2013 - removed). 2002/402/CFSP PRIMARY NAME: Zaini Zakaria (alias Ahmad). Address: Kota Bharu, Kelantan, Malaysia. Date of birth: 16.5.1967. Place of birth: Ke

In [16]:
text =  "[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. POB: Kelantan, Malaysia. Citizenship: Malaysia. Address: Kota Bharu, Kelantan, Malaysia. Additional Information: Passport number A11457974, National identification number 670516-03-5283 In custody in the Philippines as of Feb. 2008. Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [CANADA SANCTIONS - UNSTR] May 2011 - amended. Mar 2013 - removed. QI.Z.127.03. PRIMARY NAME: Zakaria,Zaini. POB; Kelantan, Malaysia. DOB: 16/05/67. Nationality: Malaysian. Passport no.: A 11457974; National identification no.: 670516-03-5283; Address: Kota Bharu, State of Kelantan, Malaysia. [EU SANCTIONS] EC 881/2002 (Apr 2013 - removed). 2002/402/CFSP PRIMARY NAME: Zaini Zakaria (alias Ahmad). Address: Kota Bharu, Kelantan, Malaysia. Date of birth: 16.5.1967. Place of birth: Kelantan, Malaysia. Nationality: Malaysian. Passport No: A11457974. National identification No: 670516-03-5283. Date of designation referred to in Article 2a (4) (b): 9.9.2003. [HONG KONG SANCTIONS - HKGAZ] G.N.6664 (14 Oct 2011) (AQ/TA split). G.N.2219 (26 April 2013 - removed). QI.Z.127.03. PRIMARY NAME:ZAINI ZAKARIA. DOB: 16 May 1967. POB: Kelantan, Malaysia. Low quality a.k.a.: Ahmad. Nationality: Malaysian. Passport no.: A 11457974. National identification no.: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [ISRAEL SANCTIONS - IMOD] Nov 2011 - addition. May 2013 - removed. Listed under Section 2 of Prohibition of Financing Terrorism Law. PRIMARY NAME: Zaini Zakaria. [SWITZERLAND SANCTIONS - SECO] SSID: 10-17045 (Sep 2003 - addition. Apr 2013 - removed). Foreign identifier: QI.Z.127.03. PRIMARY NAME: Zaini Zakaria DOB: 16 May 1967 POB: Kelantan, Malaysia Low quality a.k.a.: Ahmad Address: Kota Bharu, State of Kelantan, Malaysia Nationality: Malaysia Identification document: a) Passport No. A 11457974, Malaysia b) ID card No. 670516-03-5283, Malaysia Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun 2009. [UK SANCTIONS - UKHMT] Sep 2003 - addition. Apr 2013 - removed. PRIMARY NAME: ZAKARIA,ZAINI. DOB: 16/05/1967. POB: Kelantan, Malaysia. Nationality: Malaysian. Passport Details: A 11457974. National Identification no: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other Information: UN Ref QI.Z.127.03. Also referred to as Ahmad. Group ID: 7852. [UN SANCTIONS] UNSCR 1267 (1999) (Sep 2003 - addition. Mar 2013 - removed). QI.Z.127.03. PRIMARY NAME:ZAINI ZAKARIA. DOB: 16 May 1967. POB: Kelantan, Malaysia. Low quality a.k.a.: Ahmad. Nationality: Malaysian. Passport no.: A 11457974. National identification no.: 670516-03-5283. Address: Kota Bharu, State of Kelantan, Malaysia. Other information: Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [UN REPORTS - INTERPOL] 2006-25701. No longer appears on Interpol-United Nations Security Council Special Notice, subject to UN Sanctions - Al-Qaida (QI.Z.127.03.). [USA SANCTIONS - OFAC] SDN Ref No 7939 - SDGT (Specially Designated Global Terrorist) (Sep 2003 - addition. Sep 2009 - amended). PRIMARY NAME: ZAKARIA, Zaini. A-1-8 Pangsapuri Sri Tanjung, Jalan Sky BS2, Bukit Sentosa Rawang, Selangor 48300, Malaysia. DOB 16 Jun 1967; alt. DOB 16 May 1967. POB Kelantan, Malaysia. Passport A11457974 (Malaysia). [SANCTIONS NOTE] This entry has been removed by a published EU regulation. [SANCTIONS HISTORY] EU (Regulation)(- Apr 2013). UKHMT (Sep 2003 - Apr 2013). UN (Sep 2003 - Mar 2013). Mar 2013 - removed from Australia, Canada, China, Jersey, New Zealand, Singapore and Ukraine sanctions lists. Apr 2013 - removed from Austria, France (Regulation), Germany (Regulation), Hong Kong (HKGAZ), India, Isle of Man, Luxembourg, Malta, Russia, Spain (Regulation) and Switzerland (SECO) sanctions lists. May 2013 - removed from Israel and Japan (JMOF) sanctions lists. Sep 2013 - removed from Monaco sanctions list. Oct 2013 - removed from Azerbaijan sanctions list. Feb 2016 - removed from South Korea (KRMSF) sanctions list. Feb 2017 - no longer appears on Liechtenstein sanctions list. [BIOGRAPHY] Alleged member {General} of Jemaa Islamiyah (JI). Reportedly recruited by al Qa'ida to pilot a plane in a second wave of 9/11 style attacks on the United States. [IDENTIFICATION] Resident of Kota Bharu, Kelantan. Abdul Matin Anol Rahmat (father-in-law). Also referred to as AHMAD. [FUNDING] Engineer. [REPORTS] May 2013 - removed by the Israeli Ministerial Committee for National Security Affairs from the Individuals sanction list. Nov 2020 - reportedly subject to export control measures related to Strategic High-Tech Commodities (SHTC) imposed by Taiwan's Bureau of Foreign Trade (BFT).[NONCONVICTION TERROR CATEGORY NOTICE] This category includes information about individuals who are reportedly being investigated for, or have been arrested or charged on suspicion of, involvement in terrorism or terror related activities. The category also includes individuals or entities identified on national or internationally recognised banning, warning or wanted lists as allegedly connected to terrorism or individuals who are reportedly connected to an organisation included on any national or international terrorism list. Inclusion in this category does not mean that an individual or entity is a terrorist or terrorist organisation or that they are involved in or connected to terrorism or terror related activity. Individuals included in this category have not been convicted of any terror related activity; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the outcome of any alleged investigation, arrest, charges or any reported connection to any terror related activity and whether such allegations are denied."


In [17]:
#with chunking 

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=400, chunk_overlap=50
)
split_docs = text_splitter.create_documents([text])
len(split_docs)

5

In [25]:
split_docs[0]

Document(page_content='[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. POB: Kelantan, Malaysia. Citizenship: Malaysia. Address: Kota Bharu, Kelantan, Malaysia. Additional Information: Passport number A11457974, National identification number 670516-03-5283 In custody in the Philippines as of Feb. 2008. Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [CANADA SANCTIONS - UNSTR] May 2011 - amended. Mar 2013 - removed. QI.Z.127.03. PRIMARY NAME: Zakaria,Zaini. POB; Kelantan, Malaysia. DOB: 16/05/67. Nationality: Malaysian. Passport no.: A 11457974; National identification no.: 670516-03-5283; Address: Kota Bharu, State of Kelantan, Malaysia. [EU SANCTIONS] EC 881/2002 (Apr 2013 - removed). 2002/402/CFSP PRIMARY NAME: Zaini Zakaria (alias Ahmad). Address: Kota Bharu, Kelantan, Malaysia. Date of birth: 16.5.1967. Place of birth: Kelantan, Malaysia. Nationality: Malaysian. Passport

In [ ]:
##HERE ANJALI

In [ ]:
#larger text, splitting, old tags
l = []
st11 = time.time()
for i in split_docs:
 st_t  = time.time()
 chnk = i
 #print(chnk)
 #response = chain.run({'entity_types' :entity_types, 'relation_types': relation_types, 'examples' : examples, 'text' : st})
 #response = chain.run(entity_types =entity_types, relation_types= relation_types, examples= examples, text = st_re)
 graph_doc = llm_transformer.convert_to_graph_documents([chnk])
 l.append(graph_doc)
 et = time.time()
 print(et-st_t)
et11 = time.time()

page_content='[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. POB: Kelantan, Malaysia. Citizenship: Malaysia. Address: Kota Bharu, Kelantan, Malaysia. Additional Information: Passport number A11457974, National identification number 670516-03-5283 In custody in the Philippines as of Feb. 2008. Review pursuant to Security Council resolution 1822 (2008) was concluded on 19 Jun. 2009. [CANADA SANCTIONS - UNSTR] May 2011 - amended. Mar 2013 - removed. QI.Z.127.03. PRIMARY NAME: Zakaria,Zaini. POB; Kelantan, Malaysia. DOB: 16/05/67. Nationality: Malaysian. Passport no.: A 11457974; National identification no.: 670516-03-5283; Address: Kota Bharu, State of Kelantan, Malaysia. [EU SANCTIONS] EC 881/2002 (Apr 2013 - removed). 2002/402/CFSP PRIMARY NAME: Zaini Zakaria (alias Ahmad). Address: Kota Bharu, Kelantan, Malaysia. Date of birth: 16.5.1967. Place of birth: Kelantan, Malaysia. Nationality: Malaysian. Passport No: A114

In [15]:
for node ina graph_doc.nodes:
    print(node)

IndexError: list index out of range

In [18]:
for rel in graph_doc[1].relationships:
    print(rel)

source=Node(id='Maung Maung Kyaw', type='Person') target=Node(id='Kaung Htet', type='Person') type='FATHER_OF'
source=Node(id='Aung Mar Myint', type='Person') target=Node(id='Lin Lett Thiri', type='Person') type='MOTHER_OF'


In [16]:
for node in graph_doc[1].nodes:
    print(node)

id='Ahmed, Yasmin' type='Person'
id='Yasmin Tariq; Fatimah Mohammad' type='Alternate Name'
id='Parties Debarred for Arms Export Control Act convictions' type='List'


In [17]:
for rel in graph_doc[1].relationships:
    print(rel)

source=Node(id='Ahmed, Yasmin', type='Person') target=Node(id='Parties Debarred for Arms Export Control Act convictions', type='List') type='IS_DEBARRED'
source=Node(id='Ahmed, Yasmin', type='Person') target=Node(id='Yasmin Tariq; Fatimah Mohammad', type='Alternate Name') type='HAS_PRIMARY_NAME'


### With 512 token size

In [18]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
splits = text_splitter.split_text(text)

In [19]:
splits

['[US DEPARTMENT OF COMMERCE] Bureau of Industry and Security - Appropriate Federal Register Citations: 73 F.R. 63678 10/27/08. Name - AHMED, YASMIN 612 BUSINESS CENTRE, MUMTAZ HASAN ROAD OFF I.I. CHUNDRIGAR ROAD, KARACHI, PK. Denied Persons List effective term 16 Oct 2008 - 16 Oct 2015. Oct 2015 - denial order expired. [US STATE DEPARTMENT] Directorate of Defence Trade Controls list of Parties Debarred for Arms Export Control Act convictions. 69 FR 17468, 04/02/2004. PRIMARY NAME - Ahmed, Yasmin (a.k.a. Yasmin Tariq; Fatimah Mohammad). [REGULATIONS HISTORY] Oct 2015 - denial order expired. [BIOGRAPHY] To be determined. [IDENTIFICATION] Naturalized US citizen. Worked in United Arab Emirates (reported 2003). Tariq Ahmed (spouse).']

In [20]:
docs1 = [Document(page_content=t) for t in splits]

docs1

[Document(page_content='[US DEPARTMENT OF COMMERCE] Bureau of Industry and Security - Appropriate Federal Register Citations: 73 F.R. 63678 10/27/08. Name - AHMED, YASMIN 612 BUSINESS CENTRE, MUMTAZ HASAN ROAD OFF I.I. CHUNDRIGAR ROAD, KARACHI, PK. Denied Persons List effective term 16 Oct 2008 - 16 Oct 2015. Oct 2015 - denial order expired. [US STATE DEPARTMENT] Directorate of Defence Trade Controls list of Parties Debarred for Arms Export Control Act convictions. 69 FR 17468, 04/02/2004. PRIMARY NAME - Ahmed, Yasmin (a.k.a. Yasmin Tariq; Fatimah Mohammad). [REGULATIONS HISTORY] Oct 2015 - denial order expired. [BIOGRAPHY] To be determined. [IDENTIFICATION] Naturalized US citizen. Worked in United Arab Emirates (reported 2003). Tariq Ahmed (spouse).')]

In [21]:
graph_doc = llm_transformer.convert_to_graph_documents(docs1)

In [22]:
for node in graph_doc[0].nodes:
    print(node)

id='Naturalized US citizen' type='Identity'
id='Denied Persons List' type='List'
id='YASMIN AHMED' type='Person'
id='Directorate of Defence Trade Controls' type='Department'
id='612 BUSINESS CENTRE, MUMTAZ HASAN ROAD OFF I.I. CHUNDRIGAR ROAD, KARACHI, PK.' type='Location'
id='United Arab Emirates' type='Country'
id='73 F.R. 63678' type='Publication'
id='US STATE DEPARTMENT' type='Organization'
id='Parties Debarred for Arms Export Control Act convictions' type='List'
id='US DEPARTMENT OF COMMERCE' type='Organization'


In [24]:
for rel in graph_doc[0].relationships:
    print(rel)

source=Node(id='US DEPARTMENT OF COMMERCE', type='Organization') target=Node(id='73 F.R. 63678', type='Publication') type='PUBLISHED_IN'
source=Node(id='YASMIN AHMED', type='Person') target=Node(id='612 BUSINESS CENTRE, MUMTAZ HASAN ROAD OFF I.I. CHUNDRIGAR ROAD, KARACHI, PK.', type='Location') type='LIVES_AT'
source=Node(id='YASMIN AHMED', type='Person') target=Node(id='Denied Persons List', type='List') type='LISTED_AS'
source=Node(id='US STATE DEPARTMENT', type='Organization') target=Node(id='Directorate of Defence Trade Controls', type='Department') type='MAINTAINS'
source=Node(id='YASMIN AHMED', type='Person') target=Node(id='Parties Debarred for Arms Export Control Act convictions', type='List') type='LISTED_AS'
source=Node(id='YASMIN AHMED', type='Person') target=Node(id='Naturalized US citizen', type='Identity') type='IDENTIFIED_AS'
source=Node(id='YASMIN AHMED', type='Person') target=Node(id='United Arab Emirates', type='Country') type='WORKED_AT'


In [ ]:
llama2 , mistral

In [ ]:
a

In [10]:
#dependencies for modifying prompts for name tags only + splitting
import pandas as pd
import json
import os

from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import WikipediaLoader
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate)
from langchain import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.schema import (SystemMessage,HumanMessage,AIMessage)
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
import tiktoken
#from langchain_groq import ChatGroq
import time
from langchain.docstore.document import Document


In [11]:
entity_types = ['person','company','DateOfBirth', 'PlaceOfBirth', 'IdentificationNumber', 'family','relative', 'alias']
relation_types = ['worksFor','isBrotherOf','isFatherOf', 'CitizenOf']

# entity_types = ['primary name','alias','also known as', 'other names', 'former name', 'previous name']
# relation_types = ['alias of','former name of']

In [12]:
system_prompt = PromptTemplate(
    template = """
    You are a top-tier algorithm designed for extracting information in structured formats.
    Your task is to identify the entities and relations requested with the user prompt, from a given text.
    You must generate the output in a JSON containing a list with JSON objects having the following keys: "head", "head_type", "relation", "tail", and "tail_type".
    The "head" key must contain the text of the extracted entity with one of the types from the provided list in the user prompt. 
    The "head_type" key must contain the type of the extracted head entity which must be one of the types from {entity_types}.
    The "relation" key must contain the type of relation between the "head" and the "tail" which must be one of the relations from {relation_types}.
    The "tail" key must represent the text of an extracted entity which is the tail of the relation, and the "tail_type" key must contain the type of the tail entity from {entity_types}. 
    Attempt to extract as many entities and relations as you can. If relation is not present then only extract under 'head' and 'head_type'.
    
    IMPORTANT NOTES:
    - Don't add any explanation and text. 
    """,
    input_variables=["entity_types","relation_types"],
)


system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)

examples = [
        {
            "text":"Adam is a software engineer in Microsoft since 2009, and last year he got an award as the Best Talent" ,    
            "head": "Adam",
            "head_type": "person",
            "relation": "worksFor",
            "tail": "Microsoft",
            "tail_type": "company"
        },
        {
            "text":"[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad." ,    
            "head": "Zaini Zakaria",
            "head_type": "person"
        },
         {
            "text":"[AUSTRALIA SANCTIONS - DFAT] No 446,a (Mar 2013 - removed). PRIMARY NAME: Zaini Zakaria. Alias: Ahmad. DOB: 16-May-67. ",
            "head":'Ahmad',
            "head_type": "alias",
            "head":"16-May-67",
            "head_type":"DateofBirth"
             
        },
    
    ]

class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Microsoft, Apple, John")
    head_type: str = Field(description="type of the extracted head entity like person, company, etc")
    relation: str = Field(description="relation between the head and the tail entities")
    tail: str = Field(description="extracted second or tail entity like Microsoft, Apple, John")
    tail_type: str = Field(description="type of the extracted tail entity like person, company, etc")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

human_prompt = PromptTemplate(
    template = """ Based on the following example, extract entities and relations from the provided text.\n\n

    Use the following entity types, don't use other entity that is not defined below:
    # ENTITY TYPES:
    {entity_types}

    Use the following relation types, don't use other relation that is not defined below:
    # RELATION TYPES:
    {relation_types}

    Below are a number of examples of text and their extracted entities and relationshhips.
    {examples}

    For the following text, generate extract entitites and relations as in the provided example.\n{format_instructions}\nText: {text}""",
    input_variables=["entity_types","relation_types","examples","text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [13]:

chat_prompt

ChatPromptTemplate(input_variables=['entity_types', 'examples', 'relation_types', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['entity_types', 'relation_types'], template='\n    You are a top-tier algorithm designed for extracting information in structured formats.\n    Your task is to identify the entities and relations requested with the user prompt, from a given text.\n    You must generate the output in a JSON containing a list with JSON objects having the following keys: "head", "head_type", "relation", "tail", and "tail_type".\n    The "head" key must contain the text of the extracted entity with one of the types from the provided list in the user prompt. \n    The "head_type" key must contain the type of the extracted head entity which must be one of the types from {entity_types}.\n    The "relation" key must contain the type of relation between the "head" and the "tail" which must be one of the relations from {relation_types}.\n    The "